In [ ]:
!pip install -r requirements-lock.txt

In [ ]:
# Cell 1: Imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import argparse, json, random, os, warnings, math
from pathlib import Path

In [ ]:
# Cell 2: Model and Tokenizer Setup
model_name  = "Qwen/Qwen-1_8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = '<|endoftext|>'

# Simplified CUDA-focused device handling
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device_map = "auto" if torch.cuda.is_available() else None

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = dtype,
    device_map = device_map,
    trust_remote_code = True
)

In [ ]:
# Cell 3: Token Definitions
'''
new_tokens = ["<O_0>", "<O_1>", "<O_2>", "<O_3>", "<O_4>",  "<O_6>",  "<O_W>"]
num_added = tokenizer._tokenize.add_tokens(new_tokens)
print(num_added)
tokenizer.add_special_tokens({"additional_special_tokens": new_tokens})
model.resize_token_embeddings(len(tokenizer))
print("Added tokens:", new_tokens)

ids = tokenizer.convert_tokens_to_ids(new_tokens)
if all(i != tokenizer.unk_token_id for i in ids):
    print("added successfully")
else:
    print("failed")

for token in new_tokens:
    if token not in tokenizer.get_vocab():
        print(f"Warning: Token {token} may not have been added properly")
'''
#use placeholder tokens for the outcomes
EXTRA_BASE = 40
OUTCOMES = ["0", "1", "2", "3", "4", "6", "WICKET"]

OUTCOME2TOK = {o:f"<|extra_{EXTRA_BASE+i}|>" for i,o in enumerate(OUTCOMES)}
TOK2OUTCOME = {v:k for k,v in OUTCOME2TOK.items()}

for k,v in OUTCOME2TOK.items():
    print(f" {k:7s} -> {v}")

#sanity check for tokenizer working
for tok in  OUTCOME2TOK.values():
    ids = tokenizer(tok)["input_ids"]
    #print(tok,ids)
    assert len(ids) == 1, f"{tok} splits into multiple tokens"


In [ ]:
# Cell 4: Dataset Building Function
def build_dataset(jsonl_path: str, tokenizer, max_len: int =64):

    raw_ds = load_dataset("json", data_files=jsonl_path, split = "train")
    def _to_features(example):
        tgt_tok = OUTCOME2TOK[example["target"]]
        text = example["prompt"].rstrip() + " " + tgt_tok
        enc = tokenizer(text, truncation= True, padding = "max_length", max_length = max_len)
        # Add labels for training (same as input_ids, but mask padding tokens)
        enc["labels"] = [
            token_id if token_id != tokenizer.pad_token_id else -100 
            for token_id in enc["input_ids"]
        ]
        return enc
    return raw_ds.map(_to_features, remove_columns= raw_ds.column_names)

# ds = build_dataset('/Users/aryamangupta/CricML/llm-finetune/cricket_prompts.jsonl', tokenizer, max_len=128)
# example = ds[0]
# print(example["input_ids"])
# print("attention mask:", example["attention_mask"])
# print ("labels", example["labels"])

In [ ]:
# Cell 5: LoRA Configuration
#setup LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules= "all-linear", #["c_attn", "c_proj", "w1", "w2"],
    lora_dropout=0.0,
    bias="none"
)
model = get_peft_model(model, peft_config)

print(f"Model type: {type(model).__name__}")
model.print_trainable_parameters()

In [ ]:
# Cell 6: Training Function
#training loop
def train_qwen (model, dataset, out_dir, epochs:int, batch: int):
    args = TrainingArguments(
        output_dir= out_dir,
        per_device_train_batch_size= batch,
        num_train_epochs= epochs,
        gradient_accumulation_steps = 4,
        learning_rate= 3e-4,
        fp16 = torch.cuda.is_available(),
        logging_steps= 25,
        save_strategy= "epoch",
        report_to= "none",
        dataloader_num_workers=2
    )
    trainer = Trainer(model=model, args=args, train_dataset= dataset)
    trainer.train() 
    trainer.save_model(out_dir)
    tokenizer.save_pretrained(out_dir)
    return trainer

In [ ]:
# Cell 7: Prediction Function
def predict_dist(prompt:str, model, tokenizer):
    model.eval()
    ids = tokenizer(prompt, return_tensors = "pt").to(model.device)
    with torch.no_grad():
        logits = model(**ids).logits[0, -1, :]
    sel = tokenizer.convert_tokens_to_ids(list(OUTCOME2TOK.values()))
    probs = torch.softmax(logits[sel], dim =-1).cpu().tolist()
    return {o: p for o,p in zip(OUTCOMES, probs)}

#test
# input_ids = tokenizer("Vaibhav Arora bowling to Tristan Stubbs", return_tensors = "pt").input_ids.to(model.device)
# output = model.generate(input_ids)
# print (tokenizer.decode(output[0], skip_special_tokens = True))    

In [ ]:
# Cell 8: Training Execution
# Set your parameters here
data_path = "cricket_prompts.jsonl"  # Set to your JSONL file path
epochs = 3
batch_size = 2
output_dir = "qwen-cricket-peft"

if os.path.exists(data_path):
    ds = build_dataset(data_path, tokenizer)
    qwen_trainer = train_qwen(model, ds, output_dir, epochs, batch_size)
    
    #demo
    demo = "M Theekshana bowling to YBK Jaiswal, 7/0 after 1.4 overs, 1st innings, Pallekele International Cricket Stadium, 2024-07-30"
    dist = predict_dist (demo,model, tokenizer)
    for k,v in dist.items():
        print(f" {k:7s} : {v:.3f}")
else:
    print("Set data_path to your JSONL file to start training")